In [45]:
import networkx as nx
from multiprocessing import cpu_count
import torch_geometric.utils as pyg_utils
import torch_geometric.transforms as T
from gensim.models import Word2Vec
Gr = nx.DiGraph()
Gr1 = nx.DiGraph()
with open('Datasets/1992to1994.txt', 'r') as f:
    lines = f.readlines()

with open('Datasets/1994to1998.txt','r') as f:
    lines1 = f.readlines()
edges = []
count = 0

for line in lines:
    if count < 4:
        count += 1
    else:
        linesp = line.strip().split()
        edges.append((int(linesp[0]), int(linesp[1])))
        count+=1

Gr.add_edges_from(edges)
edges = []
count = 0

for line in lines1:
    if count < 4:
        count += 1
    else:
        linesp = line.strip().split()
        edges.append((int(linesp[0]), int(linesp[1])))
        count+=1

Gr1.add_edges_from(edges)
for edge in Gr.edges():
	Gr[edge[0]][edge[1]][1] = 1
for edge in Gr1.edges():
	Gr1[edge[0]][edge[1]][1] = 1
# now we have our graph Gr
p=1
q=1
dimensions=7
num_walks=10
walk_length=80
window_size=10
iter=1
workers=cpu_count()
import numpy as np
import random
class Graph():
	def __init__(self, nx_G, is_directed, p, q):
		self.G = nx_G
		self.is_directed = is_directed
		self.p = p
		self.q = q

	def node2vec_walk(self, walk_length, start_node):
		G = self.G
		alias_nodes = self.alias_nodes
		alias_edges = self.alias_edges

		walk = [start_node]

		while len(walk) < walk_length:
			cur = walk[-1]
			cur_nbrs = sorted(G.neighbors(cur))
			if len(cur_nbrs) > 0:
				if len(walk) == 1:
					walk.append(cur_nbrs[alias_draw(alias_nodes[cur][0], alias_nodes[cur][1])])
				else:
					prev = walk[-2]
					next = cur_nbrs[alias_draw(alias_edges[(prev, cur)][0], 
						alias_edges[(prev, cur)][1])]
					walk.append(next)
			else:
				break

		return walk

	def simulate_walks(self, num_walks, walk_length):
		G = self.G
		walks = []
		nodes = list(G.nodes())
		print('Walk iteration:')
		for walk_iter in range(num_walks):
			print(str(walk_iter+1), '/', str(num_walks))
			random.shuffle(nodes)
			for node in nodes:
				walks.append(self.node2vec_walk(walk_length=walk_length, start_node=node))

		return walks

	def get_alias_edge(self, src, dst):
		G = self.G
		p = self.p
		q = self.q

		unnormalized_probs = []
		for dst_nbr in sorted(G.neighbors(dst)):
			if dst_nbr == src:
				unnormalized_probs.append(G[dst][dst_nbr][1]/p)
			elif G.has_edge(dst_nbr, src):
				unnormalized_probs.append(G[dst][dst_nbr][1])
			else:
				unnormalized_probs.append(G[dst][dst_nbr][1]/q)
		norm_const = sum(unnormalized_probs)
		normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]

		return alias_setup(normalized_probs)

	def preprocess_transition_probs(self):
		G = self.G
		is_directed = self.is_directed

		alias_nodes = {}
		for node in G.nodes():
			unnormalized_probs = [G[node][nbr][1] for nbr in sorted(G.neighbors(node))]
			norm_const = sum(unnormalized_probs)
			normalized_probs =  [float(u_prob)/norm_const for u_prob in unnormalized_probs]
			alias_nodes[node] = alias_setup(normalized_probs)

		alias_edges = {}
		triads = {}

		if is_directed:
			for edge in G.edges():
				alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
		else:
			for edge in G.edges():
				alias_edges[edge] = self.get_alias_edge(edge[0], edge[1])
				alias_edges[(edge[1], edge[0])] = self.get_alias_edge(edge[1], edge[0])

		self.alias_nodes = alias_nodes
		self.alias_edges = alias_edges

		return


def alias_setup(probs):
        K = len(probs)
        q = np.zeros(K)
        J = np.zeros(K, dtype=float)

        smaller = []
        larger = []
        for kk, prob in enumerate(probs):
            q[kk] = K*prob
            if q[kk] < 1.0:
                smaller.append(kk)
            else:
                larger.append(kk)

        while len(smaller) > 0 and len(larger) > 0:
            small = smaller.pop()
            large = larger.pop()

            J[small] = large
            q[large] = q[large] + q[small] - 1.0
            if q[large] < 1.0:
                smaller.append(large)
            else:
                larger.append(large)

        return J, q

def alias_draw(J, q):
        K = len(J)

        kk = int(np.floor(np.random.rand()*K))
        if np.random.rand() < q[kk]:
            return kk
        else:
            return J[kk]
        
G=Graph(Gr, True, p, q)
G.preprocess_transition_probs()
walks = G.simulate_walks(num_walks, walk_length)
walks = [list(map(str, walk)) for walk in walks]
model = Word2Vec(walks, vector_size=dimensions, window=window_size, min_count=0, sg=1, workers=workers, epochs=iter)
emb_mappings = model.wv
G1=Graph(Gr1, True, p, q)
G1.preprocess_transition_probs()
walks1 = G1.simulate_walks(num_walks, walk_length)
walks1 = [list(map(str, walk)) for walk in walks1]
model1 = Word2Vec(walks1, vector_size=dimensions, window=window_size, min_count=0, sg=1, workers=workers, epochs=iter)
emb_mappings1 = model1.wv

Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10
Walk iteration:
1 / 10
2 / 10
3 / 10
4 / 10
5 / 10
6 / 10
7 / 10
8 / 10
9 / 10
10 / 10


In [46]:
data=pyg_utils.convert.from_networkx(Gr)
transform=T.RandomLinkSplit(add_negative_train_samples=True)
data_train,data_val,data_test = transform(data)
data1=pyg_utils.convert.from_networkx(Gr1)
positive_edge_train = []
negative_edge_train = []
positive_edge_val = []
negative_edge_val = []
positive_edge_test = []
negative_edge_test = []
for i in range(0,len(data_train.edge_label)):
    if(data_train.edge_label[i]==1):
        positive_edge_train.append(data_train.edge_label_index[:,i].tolist())
    else:
        negative_edge_train.append(data_train.edge_label_index[:,i].tolist())
for i in range(0,len(data_val.edge_label)):
    if(data_val.edge_label[i]==1):
        positive_edge_val.append(data_val.edge_label_index[:,i].tolist())
    else:
        negative_edge_val.append(data_val.edge_label_index[:,i].tolist())
for i in range(0,len(data_test.edge_label)):
    if(data_test.edge_label[i]==1):
        positive_edge_test.append(data_test.edge_label_index[:,i].tolist())
    else:
        negative_edge_test.append(data_test.edge_label_index[:,i].tolist())

In [47]:
# Link prediction using GNN node embeddings
import numpy as np
import pandas as pd
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import average_precision_score,roc_auc_score
from sklearn.preprocessing import StandardScaler

In [48]:
emb_list = []
for node_index in range(0, data.num_nodes):
    # node_str = str(node_index)
    node_emb = emb_mappings[node_index]
    emb_list.append(node_emb)
emb_matrix = np.vstack(emb_list)
print(data1)
emb_list1 = []
for node_index in range(0, data1.num_nodes):
    # node_str = str(node_index)
    node_emb = emb_mappings1[node_index]
    emb_list1.append(node_emb)
emb_matrix1 = np.vstack(emb_list1)

Data(edge_index=[2, 140382], 1=[140382], num_nodes=17362)


In [49]:
def get_edge_embeddings(edge_list,choice):
    embs = []
    if(choice==0):
        for edge in edge_list:
            node1 = edge[0]
            node2 = edge[1]
            emb1 = emb_matrix[node1]
            emb2 = emb_matrix[node2]
            edge_emb = np.multiply(emb1, emb2)
            embs.append(edge_emb)
    else:
        for edge in edge_list:
            node1 = edge[0]
            node2 = edge[1]
            emb1 = emb_matrix1[node1]
            emb2 = emb_matrix1[node2]
            edge_emb = np.multiply(emb1, emb2)
            embs.append(edge_emb)
    embs = np.array(embs)
    return embs

In [50]:
pos_train_edge_embs = get_edge_embeddings(positive_edge_train,0)
neg_train_edge_embs = get_edge_embeddings(negative_edge_train,0)
train_edge_embs = np.concatenate([pos_train_edge_embs, neg_train_edge_embs])

# Create train-set edge labels: 1 = real edge, 0 = false edge
train_edge_labels = np.concatenate([np.ones(len(positive_edge_train)), np.zeros(len(negative_edge_train))])

# Val-set edge embeddings, labels
pos_val_edge_embs = get_edge_embeddings(positive_edge_val,0)
neg_val_edge_embs = get_edge_embeddings(negative_edge_val,0)
val_edge_embs = np.concatenate([pos_val_edge_embs, neg_val_edge_embs])
val_edge_labels = np.concatenate([np.ones(len(positive_edge_val)), np.zeros(len(negative_edge_val))])

# Test-set edge embeddings, labels
pos_test_edge_embs = get_edge_embeddings(positive_edge_test,0)
neg_test_edge_embs = get_edge_embeddings(negative_edge_test,0)
test_edge_embs = np.concatenate([pos_test_edge_embs, neg_test_edge_embs])

# Create val-set edge labels: 1 = real edge, 0 = false edge
test_edge_labels = np.concatenate([np.ones(len(positive_edge_test)), np.zeros(len(negative_edge_test))])

In [51]:
edge_classifier = LogisticRegression(random_state=0)
edge_classifier.fit(train_edge_embs, train_edge_labels)


data=pyg_utils.convert.from_networkx(Gr1)
transform=T.RandomLinkSplit(add_negative_train_samples=True)
data_train,data_val,data_test = transform(data)

positive_edge_train = []
negative_edge_train = []
positive_edge_val = []
negative_edge_val = []
positive_edge_test = []
negative_edge_test = []
for i in range(0,len(data_train.edge_label)):
    if(data_train.edge_label[i]==1):
        positive_edge_train.append(data_train.edge_label_index[:,i].tolist())
    else:
        negative_edge_train.append(data_train.edge_label_index[:,i].tolist())
for i in range(0,len(data_val.edge_label)):
    if(data_val.edge_label[i]==1):
        positive_edge_val.append(data_val.edge_label_index[:,i].tolist())
    else:
        negative_edge_val.append(data_val.edge_label_index[:,i].tolist())
for i in range(0,len(data_test.edge_label)):
    if(data_test.edge_label[i]==1):
        positive_edge_test.append(data_test.edge_label_index[:,i].tolist())
    else:
        negative_edge_test.append(data_test.edge_label_index[:,i].tolist())
        

pos_val_edge_embs = get_edge_embeddings(positive_edge_val,1)
neg_val_edge_embs = get_edge_embeddings(negative_edge_val,1)
val_edge_embs = np.concatenate([pos_val_edge_embs, neg_val_edge_embs])
val_edge_labels = np.concatenate([np.ones(len(positive_edge_val)), np.zeros(len(negative_edge_val))])

# Test-set edge embeddings, labels
pos_test_edge_embs = get_edge_embeddings(positive_edge_test,1)
neg_test_edge_embs = get_edge_embeddings(negative_edge_test,1)
test_edge_embs = np.concatenate([pos_test_edge_embs, neg_test_edge_embs])

# Create val-set edge labels: 1 = real edge, 0 = false edge
test_edge_labels = np.concatenate([np.ones(len(positive_edge_test)), np.zeros(len(negative_edge_test))])


val_preds = edge_classifier.predict_proba(val_edge_embs)[:, 1]
val_roc = roc_auc_score(val_edge_labels, val_preds)
val_ap = average_precision_score(val_edge_labels, val_preds)

test_preds = edge_classifier.predict_proba(test_edge_embs)[:, 1]
test_roc = roc_auc_score(test_edge_labels, test_preds)
test_ap = average_precision_score(test_edge_labels, test_preds)

print ('node2vec Validation ROC score: ', str(val_roc))
print ('node2vec Validation AP score: ', str(val_ap))
print ('node2vec Test ROC score: ', str(test_roc))
print ('node2vec Test AP score: ', str(test_ap))

node2vec Validation ROC score:  0.6531917107699512
node2vec Validation AP score:  0.6648211930307945
node2vec Test ROC score:  0.6524833261736136
node2vec Test AP score:  0.6637982342578719
